In [0]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
!pip install pytorchcv

     |████████████████████████████████| 368kB 5.0MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
import fastai
from fastai.vision import *
from pytorchcv.model_provider import get_model as ptcv_get_model
import warnings
warnings.filterwarnings('ignore')
print(fastai.__version__)

1.0.59


In [6]:
path = Path("/content/drive/My Drive/dla-project/data/")
path.ls()

[PosixPath('/content/drive/My Drive/dla-project/data/test'),
 PosixPath('/content/drive/My Drive/dla-project/data/train'),
 PosixPath('/content/drive/My Drive/dla-project/data/test_single')]

In [0]:
inference_effnet_single = load_learner(path, file=path/'train/effnet-b1-c98.pkl', test=ImageList.from_folder(path/"test_single"))
inference_incept_single = load_learner(path, file=path/'train/incept-v3-c95.pkl', test=ImageList.from_folder(path/"test_single"))


In [8]:
test_files = path/"test_single"
test_name_list = os.listdir(test_files)
test_name_list[:5]

['test_high_capacity_1.jpg',
 'test_high_capacity_2.jpg',
 'test_no_turbines_med_potential_2.jpg',
 'test_high_capacity_3.jpg',
 'test_no_turbines_no_potential_1.jpg']

In [0]:
import time

In [10]:
inference_effnet_single.model.eval()
eff_preds = {}
start = time.time()
for i in range(0, len(test_name_list)):
  img_name = str(test_name_list[i])
  img = inference_effnet_single.data.test_dl.dl.dataset.x[i]
  p = inference_effnet_single.predict(img)
  eff_preds[img_name] = str(p[0]); 
end = time.time()
pred_list = list(eff_preds.items())
print("elapsed: ", end - start)

results = pd.DataFrame(pred_list, columns=["file_name", "id"])
print(results)

elapsed:  49.95131015777588
                                file_name                          id
0                test_high_capacity_1.jpg      turbines_high_capacity
1                test_high_capacity_2.jpg      turbines_high_capacity
2    test_no_turbines_med_potential_2.jpg   no_turbines_med_potential
3                test_high_capacity_3.jpg      turbines_high_capacity
4     test_no_turbines_no_potential_1.jpg    no_turbines_no_potential
5                test_high_capacity_4.jpg      turbines_high_capacity
6                test_high_capacity_5.jpg      turbines_high_capacity
7                test_high_capacity_6.jpg      turbines_high_capacity
8                test_high_capacity_7.jpg      turbines_high_capacity
9                test_high_capacity_8.jpg      turbines_high_capacity
10                test_low_capacity_1.jpg   no_turbines_low_potential
11                test_low_capacity_2.jpg       turbines_low_capacity
12                test_low_capacity_3.jpg       turbines_low_c

In [11]:
inference_incept_single.model.eval()
incept_preds = {}
start = time.time()
for i in range(0, len(test_name_list)):
  img_name = str(test_name_list[i])
  img = inference_incept_single.data.test_dl.dl.dataset.x[i]
  p = inference_incept_single.predict(img)
  incept_preds[img_name] = str(p[0]); 
end = time.time()
pred_list = list(incept_preds.items())
print("elapsed: ", end - start)

results = pd.DataFrame(pred_list, columns=["file_name", "id"])
print(results)


elapsed:  35.5715389251709
                                file_name                          id
0                test_high_capacity_1.jpg    turbines_medium_capacity
1                test_high_capacity_2.jpg      turbines_high_capacity
2    test_no_turbines_med_potential_2.jpg    turbines_medium_capacity
3                test_high_capacity_3.jpg    turbines_medium_capacity
4     test_no_turbines_no_potential_1.jpg    no_turbines_no_potential
5                test_high_capacity_4.jpg      turbines_high_capacity
6                test_high_capacity_5.jpg      turbines_high_capacity
7                test_high_capacity_6.jpg    turbines_medium_capacity
8                test_high_capacity_7.jpg    turbines_medium_capacity
9                test_high_capacity_8.jpg    turbines_medium_capacity
10                test_low_capacity_1.jpg       turbines_low_capacity
11                test_low_capacity_2.jpg       turbines_low_capacity
12                test_low_capacity_3.jpg       turbines_low_ca